In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import glob
import os

In [ ]:
%matplotlib inline

# 1d case

In [ ]:
!ls 1d

In [ ]:
fs_imex_all = []
fs_ce_all = []
kappas = [1e-3, 1e-4, 1e-5]
for n in range(3, 3+len(kappas)):
    files_imex = sorted(glob.glob(f'1d/data_imex_*_kappa1em{n}.hdf5'), key=os.path.getmtime)
    fs_imex = []
    for f in files_imex:
        fs_imex.append(h5py.File(f, 'r'))
    fs_imex_all.append(fs_imex)
    files_ce = sorted(glob.glob(f'1d/data_ce_*_kappa1em{n}.hdf5'), key=os.path.getmtime)
    fs_ce = []
    for f in files_ce:
        fs_ce.append(h5py.File(f, 'r'))
    fs_ce_all.append(fs_ce)

In [ ]:
print(len(fs_imex_all[0]), len(fs_ce_all[0]))
print(len(fs_imex_all[1]), len(fs_ce_all[1]))
print(len(fs_imex_all[2]), len(fs_ce_all[2]))

In [ ]:
for n, kappa in enumerate(kappas):
    fig, axes = plt.subplots(4, 2, figsize=(6, 10))
    for i in range(len(fs_imex_all[n])-4, len(fs_imex_all[n])):
        axi = i+4-len(fs_imex_all[n])
        axes[axi, 0].plot(fs_imex_all[n][i]['Domain/x'], fs_imex_all[n][i]['Primitive/T'][:], label='IMEX')
        axes[axi, 0].plot(fs_ce_all[n][i]['Domain/x'], fs_ce_all[n][i]['Primitive/T'][:], label='CE')
        axes[axi, 0].legend()
        axes[axi, 0].set_title(f"nx = {fs_imex_all[n][i]['Domain'].attrs['nx'][0]}")
        axes[axi, 0].set_ylabel(r"$T$")
        axes[axi, 0].set_xlim(0, 1)
        axes[axi, 0].set_ylim(0, 1)
        axes[axi, 1].plot(fs_imex_all[n][i]['Domain/x'], fs_imex_all[n][i]['Primitive/T'][:]-fs_ce_all[n][i]['Primitive/T'][:], label='IMEX')
        axes[axi, 1].set_title("IMEX - CE")
        axes[axi, 1].set_xlim(0, 1)
        axes[axi, 1].yaxis.tick_right()
    axes[-1, 0].set_xlabel(r"$x$")
    axes[-1, 1].set_xlabel(r"$x$")
    fig.suptitle(fr"$\kappa=10^{{-{3+n}}}$")
    fig.tight_layout()
    plt.savefig(f"solutions_k{3+n}.png", bbox_inches='tight')
    plt.show()

Needs self-convergence tests here as well.

In [ ]:
def getCauchyConvergence(data, resolutions):
    cauchyerrors = []
    for i_r_c, r in enumerate(resolutions[1:]):
        i_r = i_r_c + 1
        data1 = data[i_r]['Primitive/T'][:]
        data2 = np.repeat(data[i_r_c]['Primitive/T'][:], 2)
        cauchyerrors.append(np.linalg.norm(data1-data2, 2) / r)
    return cauchyerrors

In [ ]:
fit_ranges = [[0, 5],
              [1, 7],
              [3, 7]]
for n in range(len(fs_imex_all)):
    resolutions = []
    dxs = []
    for f in fs_imex_all[n]:
        resolutions.append(f['Domain'].attrs['nx'][0])
        dxs.append(f['Domain'].attrs['dx'][0])
    cauchyerrors = getCauchyConvergence(fs_imex_all[n], resolutions)
    p = np.polyfit(np.log(dxs[fit_ranges[n][0]+1:fit_ranges[n][1]+1]),
                   np.log(cauchyerrors[fit_ranges[n][0]:fit_ranges[n][1]]),
                   1)
    plt.figure(figsize=(6, 4))
    plt.loglog(dxs[1:], cauchyerrors, 'kx', label='Data')
    plt.loglog(dxs[1+fit_ranges[n][0]:], np.exp(p[1])*dxs[1+fit_ranges[n][0]:]**p[0], label=rf"Fit slope {p[0]:.1f}")
    plt.xlabel(r"$\Delta x$")
    plt.ylabel(r"Convergence error")
    plt.title(fr"IMEX: $\kappa=10^{{-{3+n}}}$")
    plt.legend()
    plt.savefig(f"self_convergence_imex_k{3+n}.png")
    plt.show()


In [ ]:
fit_ranges = [[0, 5],
              [1, 7],
              [3, 7]]
for n in range(len(fs_ce_all)):
    resolutions = []
    dxs = []
    for f in fs_ce_all[n]:
        resolutions.append(f['Domain'].attrs['nx'][0])
        dxs.append(f['Domain'].attrs['dx'][0])
    cauchyerrors = getCauchyConvergence(fs_ce_all[n], resolutions)
    p = np.polyfit(np.log(dxs[fit_ranges[n][0]+1:fit_ranges[n][1]+1]),
                   np.log(cauchyerrors[fit_ranges[n][0]:fit_ranges[n][1]]),
                   1)
    plt.figure(figsize=(6,4))
    plt.loglog(dxs[1:], cauchyerrors, 'kx', label='Data')
    plt.loglog(dxs[1+fit_ranges[n][0]:], np.exp(p[1])*dxs[1+fit_ranges[n][0]:]**p[0], label=rf"Fit slope {p[0]:.1f}")
    plt.xlabel(r"$\Delta x$")
    plt.ylabel(r"Convergence error")
    plt.title(fr"CE: $\kappa=10^{{-{3+n}}}$")
    plt.legend()
    plt.savefig(f"self_convergence_ce_k{3+n}.png")
    plt.show()


So, we are seeing self-convergence of each individually.

Check convergence of CE to full model by comparing one against the other:

In [ ]:
def getCauchyConvergence_twomodels(model1, model2, resolutions):
    cauchyerrors = []
    for i_r, r in enumerate(resolutions):
        data1 = model1[i_r]['Primitive/T'][:]
        data2 = model2[i_r]['Primitive/T'][:]
        cauchyerrors.append(np.linalg.norm(data1-data2, 2) / r)
    return cauchyerrors

In [ ]:
fit_ranges = [[1, 6],
              [2, 8],
              [4, 8]]
plot_ranges = [[0, 6],
               [1, 8],
               [2, 8]]
for n in range(len(fs_ce_all)):
    resolutions = []
    dxs = []
    for f in fs_ce_all[n]:
        resolutions.append(f['Domain'].attrs['nx'][0])
        dxs.append(f['Domain'].attrs['dx'][0])
    cauchyerrors = getCauchyConvergence_twomodels(fs_imex_all[n],
                                                  fs_ce_all[n],
                                                  resolutions)
    p = np.polyfit(np.log(dxs[fit_ranges[n][0]:fit_ranges[n][1]]),
                   np.log(cauchyerrors[fit_ranges[n][0]:fit_ranges[n][1]]),
                   1)
    plt.figure(figsize=(6,4))
    plt.loglog(dxs[plot_ranges[n][0]:], cauchyerrors[plot_ranges[n][0]:], 'kx', label='Data')
    plt.loglog(dxs[fit_ranges[n][0]:], np.exp(p[1])*dxs[fit_ranges[n][0]:]**p[0], label=rf"Fit slope {p[0]:.1f}")
    plt.xlabel(r"$\Delta x$")
    plt.ylabel(r"Convergence error")
    plt.title(fr"CE$\to$IMEX: $\kappa=10^{{-{3+n}}}$")
    plt.legend()
    plt.savefig(f"convergence_ce_to_imex_k{3+n}.png")
    plt.show()


Because of the extreme smearing at low resolution, low $\kappa$, not all points are plotted.

So, the convergence plots show that for these values of $\kappa$ it is impossible to distinguish *model error* (from the Chapman-Enskog model) from *numerical error* (due to finite difference effects).

Check how the solutions at different $\kappa$ look at the end.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(10,5))
for n in range(len(fs_imex_all)):
    kappa = fs_imex_all[n][-1]['Optional'].attrs['kappa'][0]
    endTime = fs_imex_all[n][-1].attrs['t'][0]
    axes[0].plot(fs_imex_all[n][-1]['Domain/x'],
                 fs_imex_all[n][-1]['Primitive/T'], label=rf"$\kappa={kappa:.1e}, t={endTime}$")
    axes[1].plot(fs_ce_all[n][-1]['Domain/x'],
                 fs_ce_all[n][-1]['Primitive/T'], label=rf"$\kappa={kappa:.1e}, t={endTime}$")
axes[0].set_title('IMEX')
axes[1].set_title('CE')
for ax in axes:
    ax.legend()
    ax.set_xlabel(r"$x$")
    ax.set_ylabel(r"$T$")
fig.tight_layout()
plt.show()

If we went to smaller $\kappa$ we'd have to evolve a very long time, introducing large numerical errors from the integration.